In [ ]:
import numpy as np
import statsmodels.api as sm
from pandas.stats.api import ols
from sklearn.linear_model import LinearRegression

import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, PercentChange, SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.factset import Fundamentals as FstFundamentals
# from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline import factors, filters, classifiers
import alphalens
import quantopian.pipeline.data.factset.estimates as fe


def signalize(df):
    return ((df.rank() - 0.5)/df.count()).replace(np.nan,0.5)

def transform(df, field, multiplier=1):
    return signalize(multiplier*df[field])

class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = close[-1] * shares[-1]

class LogMarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = np.log(close[-1] * shares[-1])

class Volatility(CustomFactor):  
    inputs = [USEquityPricing.close]  
    window_length = 25  
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0) * math.sqrt(252)

class Industry(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]

        
class IndustryGroup(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_group_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]


class Sector(CustomFactor):  
    inputs = [Fundamentals.morningstar_sector_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]

class Previous(CustomFactor):  
    # Returns value of input x trading days ago where x is the window_length  
    # Both the inputs and window_length must be specified as there are no defaults

    def compute(self, today, assets, out, inputs):  
        out[:] = inputs[0]


        
base_universe = QTradableStocksUS()

mkt_cap = MarketCap(mask = base_universe)
log_mkt_cap = LogMarketCap(mask = base_universe)
vol = Volatility(mask = base_universe)

industry = Industry(mask = base_universe)
industryGroup = IndustryGroup(mask = base_universe)
sector = Sector(mask = base_universe)

# f18 = f3.demean(mask = f13.notnan(), groupby=classifiers.fundamentals.Sector())

fq1_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 1)
fq1_eps_std = fq1_eps_cons.std_dev.latest
fq1_eps_std_exsec = fq1_eps_cons.std_dev.latest.demean(mask = fq1_eps_std.notnan(), groupby=classifiers.fundamentals.Sector())
fq1_eps_mean = fq1_eps_cons.mean.latest

# fq1_eps_median = fq1_eps_cons.median.latest
# fq1_eps_high = fq1_eps_cons.high.latest
# fq1_eps_low = fq1_eps_cons.low.latest

fq1_eps_up = fq1_eps_cons.up.latest
fq1_eps_dwn = fq1_eps_cons.down.latest
fq1_eps_num_est = fq1_eps_cons.num_est.latest

# fq1_eps_change_pct = PercentChange(inputs=[fq1_eps_cons.mean], window_length=5)
# fq1_eps_high_change_pct = PercentChange(inputs=[fq1_eps_cons.high], window_length=5)


fq1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 1)
fq1_sales_std = fq1_sales_cons.std_dev.latest
fq1_sales_mean = fq1_sales_cons.mean.latest
# fq1_sales_up = fq1_sales_cons.up.latest
# fq1_sales_dwn = fq1_sales_cons.down.latest
# fq1_sales_num_est = fq1_sales_cons.num_est.latest

# fq1_cfps_cons = fe.PeriodicConsensus.slice('CFPS', 'qf', 1)
# fq1_cfps_std = fq1_cfps_cons.std_dev.latest
# fq1_cfps_mean = fq1_cfps_cons.mean.latest
# fq1_cfps_up = fq1_cfps_cons.up.latest
# fq1_cfps_dwn = fq1_cfps_cons.down.latest
# fq1_cfps_num_est = fq1_cfps_cons.num_est.latest


# fq0_cfps_cons = fe.PeriodicConsensus.slice('CFPS', 'qf', 0)
# fq0_cfps_act = fe.Actuals.slice('CFPS', 'qf', 0)
# fq0_cfps_cons_mean = fq0_cfps_cons.mean.latest
# fq0_cfps_act_value = fq0_cfps_act.actual_value.latest
# fq0_cfps_surprise = (fq0_cfps_act_value - fq0_cfps_cons_mean)/fq0_cfps_cons_mean

fq0_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 0)
fq0_eps_act = fe.Actuals.slice('EPS', 'qf', 0)
fq0_eps_cons_mean = fq0_eps_cons.mean.latest
fq0_eps_act_value = fq0_eps_act.actual_value.latest

fq0_eps_surprise = (fq0_eps_act_value - fq0_eps_cons_mean)/fq0_eps_cons_mean

fq0_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 0)
fq0_sales_act = fe.Actuals.slice('SALES', 'qf', 0)
fq0_sales_cons_mean = fq0_sales_cons.mean.latest
fq0_sales_act_value = fq0_sales_act.actual_value.latest

fq0_sales_surprise = (fq0_sales_act_value - fq0_sales_cons_mean)/fq0_sales_cons_mean

#Long term
# price_tgt_cons = fe.LongTermConsensus.slice('PRICE_TGT')
# eps_gr_cons = fe.LongTermConsensus.slice('EPS_LTG')

# Get the latest mean consensus price target and EPS growth.
# price_tgt_mean = price_tgt_cons.mean.latest
# eps_gr_mean = eps_gr_cons.mean.latest
# price_tgt_std = price_tgt_cons.std_dev.latest
# eps_gr_std = eps_gr_cons.std_dev.latest

# Define an estimated price growth factor by taking the relative
# difference between the 6-12 month price target and yesterday's
# close price.
# yesterday_close = USEquityPricing.close.latest
# est_price_growth = (price_tgt_mean - yesterday_close) / yesterday_close

fe_rec = fe.ConsensusRecommendations

#It's actually SALES (check if below factor works with EPS)
guid_eps_q1 = fe.Guidance.slice('SALES', 'qf', 1)
guid_eps_q1_low = guid_eps_q1.low.latest
guid_eps_q1_high = guid_eps_q1.high.latest

guid_eps_std = (guid_eps_q1_high - guid_eps_q1_low)/(guid_eps_q1_low + guid_eps_q1_high)

#Quarterly Historical EPS/SALES growth - Actual

fq0_eps_act = fe.Actuals.slice('EPS', 'qf', 0).actual_value.latest
# fqn1_eps_act = fe.Actuals.slice('EPS', 'qf', -1).actual_value.latest
# fqn2_eps_act = fe.Actuals.slice('EPS', 'qf', -2).actual_value.latest
# fqn3_eps_act = fe.Actuals.slice('EPS', 'qf', -3).actual_value.latest
# fqn4_eps_act = fe.Actuals.slice('EPS', 'qf', -4).actual_value.latest
# fqn5_eps_act = fe.Actuals.slice('EPS', 'qf', -5).actual_value.latest
# fqn6_eps_act = fe.Actuals.slice('EPS', 'qf', -6).actual_value.latest
# fqn7_eps_act = fe.Actuals.slice('EPS', 'qf', -7).actual_value.latest
# fqn8_eps_act = fe.Actuals.slice('EPS', 'qf', -8).actual_value.latest

fq0_sales_act = fe.Actuals.slice('SALES', 'qf', 0).actual_value.latest
fqn1_sales_act = fe.Actuals.slice('SALES', 'qf', -1).actual_value.latest
fqn2_sales_act = fe.Actuals.slice('SALES', 'qf', -2).actual_value.latest
fqn3_sales_act = fe.Actuals.slice('SALES', 'qf', -3).actual_value.latest
fqn4_sales_act = fe.Actuals.slice('SALES', 'qf', -4).actual_value.latest
fqn5_sales_act = fe.Actuals.slice('SALES', 'qf', -5).actual_value.latest
fqn6_sales_act = fe.Actuals.slice('SALES', 'qf', -6).actual_value.latest
fqn7_sales_act = fe.Actuals.slice('SALES', 'qf', -7).actual_value.latest
fqn8_sales_act = fe.Actuals.slice('SALES', 'qf', -8).actual_value.latest


#Quarterly Consensus EPS/SALES
fq1_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 1).mean.latest
fq0_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', 0).mean.latest
fqn1_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -1).mean.latest
fqn2_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -2).mean.latest
fqn3_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -3).mean.latest
fqn4_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -4).mean.latest
fqn5_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -5).mean.latest
fqn6_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -6).mean.latest
fqn7_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -7).mean.latest
fqn8_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -8).mean.latest
fqn9_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -9).mean.latest
fqn10_eps_cons = fe.PeriodicConsensus.slice('EPS', 'qf', -10).mean.latest

fq1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 1).mean.latest
fq0_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 0).mean.latest
fqn1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -1).mean.latest
fqn2_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -2).mean.latest
fqn3_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -3).mean.latest
fqn4_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -4).mean.latest
fqn5_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -5).mean.latest
fqn6_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -6).mean.latest
fqn7_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -7).mean.latest
fqn8_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -8).mean.latest
fqn9_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -9).mean.latest
fqn10_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -10).mean.latest


# fqn1_eps_surprise = (fqn1_eps_act - fqn1_eps_cons)/fqn1_eps_cons
# fqn2_eps_surprise = (fqn2_eps_act - fqn2_eps_cons)/fqn2_eps_cons
# fqn3_eps_surprise = (fqn3_eps_act - fqn3_eps_cons)/fqn3_eps_cons
# fqn4_eps_surprise = (fqn4_eps_act - fqn4_eps_cons)/fqn4_eps_cons
# fqn5_eps_surprise = (fqn5_eps_act - fqn5_eps_cons)/fqn5_eps_cons
# fqn6_eps_surprise = (fqn6_eps_act - fqn6_eps_cons)/fqn6_eps_cons
# fqn7_eps_surprise = (fqn7_eps_act - fqn7_eps_cons)/fqn7_eps_cons
# fqn8_eps_surprise = (fqn8_eps_act - fqn8_eps_cons)/fqn8_eps_cons


fq0_sales_surprise = (fq0_sales_act - fq0_sales_cons)/fq0_sales_cons

fqn1_sales_surprise = (fqn1_sales_act - fqn1_sales_cons)/fqn1_sales_cons
fqn2_sales_surprise = (fqn2_sales_act - fqn2_sales_cons)/fqn2_sales_cons
fqn3_sales_surprise = (fqn3_sales_act - fqn3_sales_cons)/fqn3_sales_cons
fqn4_sales_surprise = (fqn4_sales_act - fqn4_sales_cons)/fqn4_sales_cons
fqn5_sales_surprise = (fqn5_sales_act - fqn5_sales_cons)/fqn5_sales_cons
fqn6_sales_surprise = (fqn6_sales_act - fqn6_sales_cons)/fqn6_sales_cons
fqn7_sales_surprise = (fqn7_sales_act - fqn7_sales_cons)/fqn7_sales_cons
fqn8_sales_surprise = (fqn8_sales_act - fqn8_sales_cons)/fqn8_sales_cons


#SALES YIELD - CONSENSUS
fq0_sales_yield_cons = fq0_sales_cons/(fq0_sales_cons - fq0_eps_cons * Fundamentals.shares_outstanding)
fqn1_sales_yield_cons = fqn1_sales_cons/(fqn1_sales_cons - fqn1_eps_cons * Fundamentals.shares_outstanding)
fqn2_sales_yield_cons = fqn2_sales_cons/(fqn2_sales_cons - fqn2_eps_cons * Fundamentals.shares_outstanding)
fqn3_sales_yield_cons = fqn3_sales_cons/(fqn3_sales_cons - fqn3_eps_cons * Fundamentals.shares_outstanding)
fqn4_sales_yield_cons = fqn4_sales_cons/(fqn4_sales_cons - fqn4_eps_cons * Fundamentals.shares_outstanding)
fqn5_sales_yield_cons = fqn5_sales_cons/(fqn5_sales_cons - fqn5_eps_cons * Fundamentals.shares_outstanding)
fqn6_sales_yield_cons = fqn6_sales_cons/(fqn6_sales_cons - fqn6_eps_cons * Fundamentals.shares_outstanding)
fqn7_sales_yield_cons = fqn7_sales_cons/(fqn7_sales_cons - fqn7_eps_cons * Fundamentals.shares_outstanding)
fqn8_sales_yield_cons = fqn8_sales_cons/(fqn8_sales_cons - fqn8_eps_cons * Fundamentals.shares_outstanding)

#Earning Yield
# fq0_eps_yield_cons = fq0_eps_cons*Fundamentals.shares_outstanding/(fq0_sales_cons - fq0_eps_cons * Fundamentals.shares_outstanding)
# fqn1_eps_yield_cons = fqn1_eps_cons*Fundamentals.shares_outstanding/(fqn1_sales_cons - fqn1_eps_cons * Fundamentals.shares_outstanding)
# fqn2_eps_yield_cons = fqn2_eps_cons*Fundamentals.shares_outstanding/(fqn2_sales_cons - fqn2_eps_cons * Fundamentals.shares_outstanding)
# fqn3_eps_yield_cons = fqn3_eps_cons*Fundamentals.shares_outstanding/(fqn3_sales_cons - fqn3_eps_cons * Fundamentals.shares_outstanding)
# fqn4_eps_yield_cons = fqn4_eps_cons*Fundamentals.shares_outstanding/(fqn4_sales_cons - fqn4_eps_cons * Fundamentals.shares_outstanding)
# fqn5_eps_yield_cons = fqn5_eps_cons*Fundamentals.shares_outstanding/(fqn5_sales_cons - fqn5_eps_cons * Fundamentals.shares_outstanding)
# fqn6_eps_yield_cons = fqn6_eps_cons*Fundamentals.shares_outstanding/(fqn6_sales_cons - fqn6_eps_cons * Fundamentals.shares_outstanding)
# fqn7_eps_yield_cons = fqn7_eps_cons*Fundamentals.shares_outstanding/(fqn7_sales_cons - fqn7_eps_cons * Fundamentals.shares_outstanding)
# fqn8_eps_yield_cons = fqn8_eps_cons*Fundamentals.shares_outstanding/(fqn8_sales_cons - fqn8_eps_cons * Fundamentals.shares_outstanding)

#GUIDANCE
guid_eps_q1_high = fe.Guidance.slice('EPS', 'qf', 1).high.latest
guid_eps_q0_high = fe.Guidance.slice('EPS', 'qf', 0).high.latest
guid_eps_qn1_high = fe.Guidance.slice('EPS', 'qf', -1).high.latest
guid_eps_qn2_high = fe.Guidance.slice('EPS', 'qf', -2).high.latest
guid_eps_qn3_high = fe.Guidance.slice('EPS', 'qf', -3).high.latest
guid_eps_qn4_high = fe.Guidance.slice('EPS', 'qf', -4).high.latest
guid_eps_qn5_high = fe.Guidance.slice('EPS', 'qf', -5).high.latest
guid_eps_qn6_high = fe.Guidance.slice('EPS', 'qf', -6).high.latest
guid_eps_qn7_high = fe.Guidance.slice('EPS', 'qf', -7).high.latest
guid_eps_qn8_high = fe.Guidance.slice('EPS', 'qf', -8).high.latest
guid_eps_qn9_high = fe.Guidance.slice('EPS', 'qf', -9).high.latest
guid_eps_qn10_high = fe.Guidance.slice('EPS', 'qf', -10).high.latest

guid_eps_q1_low = fe.Guidance.slice('EPS', 'qf', 1).low.latest
guid_eps_q0_low = fe.Guidance.slice('EPS', 'qf', 0).low.latest
guid_eps_qn1_low = fe.Guidance.slice('EPS', 'qf', -1).low.latest
guid_eps_qn2_low = fe.Guidance.slice('EPS', 'qf', -2).low.latest
guid_eps_qn3_low = fe.Guidance.slice('EPS', 'qf', -3).low.latest
guid_eps_qn4_low = fe.Guidance.slice('EPS', 'qf', -4).low.latest
guid_eps_qn5_low = fe.Guidance.slice('EPS', 'qf', -5).low.latest
guid_eps_qn6_low = fe.Guidance.slice('EPS', 'qf', -6).low.latest
guid_eps_qn7_low = fe.Guidance.slice('EPS', 'qf', -7).low.latest
guid_eps_qn8_low = fe.Guidance.slice('EPS', 'qf', -8).low.latest
guid_eps_qn9_low = fe.Guidance.slice('EPS', 'qf', -9).low.latest
guid_eps_qn10_low = fe.Guidance.slice('EPS', 'qf', -10).low.latest


guid_sales_q1_high = fe.Guidance.slice('SALES', 'qf', 1).high.latest
guid_sales_q0_high = fe.Guidance.slice('SALES', 'qf', 0).high.latest
guid_sales_qn1_high = fe.Guidance.slice('SALES', 'qf', -1).high.latest
guid_sales_qn2_high = fe.Guidance.slice('SALES', 'qf', -2).high.latest
guid_sales_qn3_high = fe.Guidance.slice('SALES', 'qf', -3).high.latest
guid_sales_qn4_high = fe.Guidance.slice('SALES', 'qf', -4).high.latest
guid_sales_qn5_high = fe.Guidance.slice('SALES', 'qf', -5).high.latest
guid_sales_qn6_high = fe.Guidance.slice('SALES', 'qf', -6).high.latest
guid_sales_qn7_high = fe.Guidance.slice('SALES', 'qf', -7).high.latest
guid_sales_qn8_high = fe.Guidance.slice('SALES', 'qf', -8).high.latest
guid_sales_qn9_high = fe.Guidance.slice('SALES', 'qf', -9).high.latest
guid_sales_qn10_high = fe.Guidance.slice('SALES', 'qf', -10).high.latest

guid_sales_q1_low = fe.Guidance.slice('SALES', 'qf', 1).low.latest
guid_sales_q0_low = fe.Guidance.slice('SALES', 'qf', 0).low.latest
guid_sales_qn1_low = fe.Guidance.slice('SALES', 'qf', -1).low.latest
guid_sales_qn2_low = fe.Guidance.slice('SALES', 'qf', -2).low.latest
guid_sales_qn3_low = fe.Guidance.slice('SALES', 'qf', -3).low.latest
guid_sales_qn4_low = fe.Guidance.slice('SALES', 'qf', -4).low.latest
guid_sales_qn5_low = fe.Guidance.slice('SALES', 'qf', -5).low.latest
guid_sales_qn6_low = fe.Guidance.slice('SALES', 'qf', -6).low.latest
guid_sales_qn7_low = fe.Guidance.slice('SALES', 'qf', -7).low.latest
guid_sales_qn8_low = fe.Guidance.slice('SALES', 'qf', -8).low.latest
guid_sales_qn9_low = fe.Guidance.slice('SALES', 'qf', -9).low.latest
guid_sales_qn10_low = fe.Guidance.slice('SALES', 'qf', -10).low.latest


#EPS Guidane Surprise
fq1_eps_guid = (guid_eps_q1_low + guid_eps_q1_high)/2
fq0_eps_guid = (guid_eps_q0_low + guid_eps_q0_high)/2
fqn1_eps_guid = (guid_eps_qn1_low + guid_eps_qn1_high)/2
fqn2_eps_guid = (guid_eps_qn2_low + guid_eps_qn2_high)/2
fqn3_eps_guid = (guid_eps_qn3_low + guid_eps_qn3_high)/2
fqn4_eps_guid = (guid_eps_qn4_low + guid_eps_qn4_high)/2
fqn5_eps_guid = (guid_eps_qn5_low + guid_eps_qn5_high)/2
fqn6_eps_guid = (guid_eps_qn6_low + guid_eps_qn6_high)/2
fqn7_eps_guid = (guid_eps_qn7_low + guid_eps_qn7_high)/2
fqn8_eps_guid = (guid_eps_qn8_low + guid_eps_qn8_high)/2

guid_eps_surprise_q1 = fq1_eps_guid - fq1_eps_cons
guid_eps_surprise_q0 = fq0_eps_guid - fq0_eps_cons
guid_eps_surprise_qn1 = fqn1_eps_guid - fqn1_eps_cons
guid_eps_surprise_qn2 = fqn2_eps_guid - fqn2_eps_cons
guid_eps_surprise_qn3 = fqn3_eps_guid - fqn3_eps_cons
guid_eps_surprise_qn4 = fqn4_eps_guid - fqn4_eps_cons
guid_eps_surprise_qn5 = fqn5_eps_guid - fqn5_eps_cons
guid_eps_surprise_qn6 = fqn6_eps_guid - fqn6_eps_cons
guid_eps_surprise_qn7 = fqn7_eps_guid - fqn7_eps_cons
guid_eps_surprise_qn8 = fqn8_eps_guid - fqn8_eps_cons


#Sales Guidance Surprise
fq1_sales_guid = (guid_sales_q1_low + guid_sales_q1_high)/2
fq0_sales_guid = (guid_sales_q0_low + guid_sales_q0_high)/2
fqn1_sales_guid = (guid_sales_qn1_low + guid_sales_qn1_high)/2
fqn2_sales_guid = (guid_sales_qn2_low + guid_sales_qn2_high)/2
fqn3_sales_guid = (guid_sales_qn3_low + guid_sales_qn3_high)/2
fqn4_sales_guid = (guid_sales_qn4_low + guid_sales_qn4_high)/2
fqn5_sales_guid = (guid_sales_qn5_low + guid_sales_qn5_high)/2
fqn6_sales_guid = (guid_sales_qn6_low + guid_sales_qn6_high)/2
fqn7_sales_guid = (guid_sales_qn7_low + guid_sales_qn7_high)/2
fqn8_sales_guid = (guid_sales_qn8_low + guid_sales_qn8_high)/2


guid_sales_surprise_q1 = fq1_sales_guid - fq1_sales_cons
guid_sales_surprise_q0 = fq0_sales_guid - fq0_sales_cons
guid_sales_surprise_qn1 = fqn1_sales_guid - fqn1_sales_cons
guid_sales_surprise_qn2 = fqn2_sales_guid - fqn2_sales_cons
guid_sales_surprise_qn3 = fqn3_sales_guid - fqn3_sales_cons
guid_sales_surprise_qn4 = fqn4_sales_guid - fqn4_sales_cons
guid_sales_surprise_qn5 = fqn5_sales_guid - fqn5_sales_cons
guid_sales_surprise_qn6 = fqn6_sales_guid - fqn6_sales_cons
guid_sales_surprise_qn7 = fqn7_sales_guid - fqn7_sales_cons
guid_sales_surprise_qn8 = fqn8_sales_guid - fqn8_sales_cons


#Sales Yield - Guidance
# fq0_sales_yield_guid = fq0_sales_guid/(fq0_sales_guid - fq0_eps_guid * Fundamentals.shares_outstanding)
# fqn1_sales_yield_guid = fqn1_sales_guid/(fqn1_sales_guid - fqn1_eps_guid * Fundamentals.shares_outstanding)
# fqn2_sales_yield_guid = fqn2_sales_guid/(fqn2_sales_guid - fqn2_eps_guid * Fundamentals.shares_outstanding)
# fqn3_sales_yield_guid = fqn3_sales_guid/(fqn3_sales_guid - fqn3_eps_guid * Fundamentals.shares_outstanding)
# fqn4_sales_yield_guid = fqn4_sales_guid/(fqn4_sales_guid - fqn4_eps_guid * Fundamentals.shares_outstanding)
# fqn5_sales_yield_guid = fqn5_sales_guid/(fqn5_sales_guid - fqn5_eps_guid * Fundamentals.shares_outstanding)
# fqn6_sales_yield_guid = fqn6_sales_guid/(fqn6_sales_guid - fqn6_eps_guid * Fundamentals.shares_outstanding)
# fqn7_sales_yield_guid = fqn7_sales_guid/(fqn7_sales_guid - fqn7_eps_guid * Fundamentals.shares_outstanding)
# fqn8_sales_yield_guid = fqn8_sales_guid/(fqn8_sales_guid - fqn8_eps_guid * Fundamentals.shares_outstanding)


fq0_sales_surprise_guid = (fq0_sales_act - fq0_sales_guid)/fq0_sales_guid
fqn1_sales_surprise_guid = (fqn1_sales_act - fqn1_sales_guid)/fqn1_sales_guid
fqn2_sales_surprise_guid = (fqn2_sales_act - fqn2_sales_guid)/fqn2_sales_guid
fqn3_sales_surprise_guid = (fqn3_sales_act - fqn3_sales_guid)/fqn3_sales_guid
fqn4_sales_surprise_guid = (fqn4_sales_act - fqn4_sales_guid)/fqn4_sales_guid
fqn5_sales_surprise_guid = (fqn5_sales_act - fqn5_sales_guid)/fqn5_sales_guid
fqn6_sales_surprise_guid = (fqn6_sales_act - fqn6_sales_guid)/fqn6_sales_guid
fqn7_sales_surprise_guid = (fqn7_sales_act - fqn7_sales_guid)/fqn7_sales_guid
fqn8_sales_surprise_guid = (fqn8_sales_act - fqn8_sales_guid)/fqn8_sales_guid


# sales_to_ev_qn8 = fqn8_sales_cons/fqn9_sales_cons
# sales_to_ev_qn7 = fqn7_sales_cons/fqn8_sales_cons
# sales_to_ev_qn6 = fqn6_sales_cons/fqn7_sales_cons
# sales_to_ev_qn5 = fqn5_sales_cons/fqn6_sales_cons
# sales_to_ev_qn4 = fqn4_sales_cons/fqn5_sales_cons
# sales_to_ev_qn3 = fqn3_sales_cons/fqn4_sales_cons
# sales_to_ev_qn2 = fqn2_sales_cons/fqn3_sales_cons
# sales_to_ev_qn1 = fqn1_sales_cons/fqn2_sales_cons
# sales_to_ev_q0 = fq0_sales_cons/fqn1_sales_cons


# sales_to_ev_q0 = sales_to_ev_q0.demean(mask = sales_to_ev_q0.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn1 = sales_to_ev_qn1.demean(mask = sales_to_ev_qn1.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn2 = sales_to_ev_qn2.demean(mask = sales_to_ev_qn2.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn3 = sales_to_ev_qn3.demean(mask = sales_to_ev_qn3.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn4 = sales_to_ev_qn4.demean(mask = sales_to_ev_qn4.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn5 = sales_to_ev_qn5.demean(mask = sales_to_ev_qn5.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn6 = sales_to_ev_qn6.demean(mask = sales_to_ev_qn6.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn7 = sales_to_ev_qn7.demean(mask = sales_to_ev_qn7.notnan(), groupby=classifiers.fundamentals.Sector())
# sales_to_ev_qn8 = sales_to_ev_qn8.demean(mask = sales_to_ev_qn8.notnan(), groupby=classifiers.fundamentals.Sector())


#Consistency of Sales Growth from Consensus vs Actual
#W
sales_shortfall_guid_q0 = (fq1_sales_cons/fq0_sales_act) 
sales_shortfall_guid_qn1 = (fq0_sales_cons/fqn1_sales_act)
sales_shortfall_guid_qn2 = (fqn1_sales_cons/fqn2_sales_act)
sales_shortfall_guid_qn3 = (fqn2_sales_cons/fqn3_sales_act)
sales_shortfall_guid_qn4 = (fqn3_sales_cons/fqn4_sales_act)
sales_shortfall_guid_qn5 = (fqn4_sales_cons/fqn5_sales_act)
sales_shortfall_guid_qn6 = (fqn5_sales_cons/fqn6_sales_act)
sales_shortfall_guid_qn7 = (fqn6_sales_cons/fqn7_sales_act)
sales_shortfall_guid_qn8 = (fqn7_sales_cons/fqn8_sales_act)


# 


pipe_my_alpha_factor = Pipeline(
    columns={
        'vol': vol,
        'log_mkt_cap': log_mkt_cap,
        'mkt_cap':mkt_cap,
        
        'guid_sales_surprise_q1':guid_sales_surprise_q1,
'guid_sales_surprise_q0':guid_sales_surprise_q0,
'guid_sales_surprise_qn1':guid_sales_surprise_qn1,
'guid_sales_surprise_qn2':guid_sales_surprise_qn2,
'guid_sales_surprise_qn3':guid_sales_surprise_qn3,
'guid_sales_surprise_qn4':guid_sales_surprise_qn4,
'guid_sales_surprise_qn5':guid_sales_surprise_qn5,
'guid_sales_surprise_qn6':guid_sales_surprise_qn6,
'guid_sales_surprise_qn7':guid_sales_surprise_qn7,
'guid_sales_surprise_qn8':guid_sales_surprise_qn8,
        
#         'guid_eps_surprise_q1':guid_eps_surprise_q1,
# 'guid_eps_surprise_q0':guid_eps_surprise_q0,
# 'guid_eps_surprise_qn1':guid_eps_surprise_qn1,
# 'guid_eps_surprise_qn2':guid_eps_surprise_qn2,
# 'guid_eps_surprise_qn3':guid_eps_surprise_qn3,
# 'guid_eps_surprise_qn4':guid_eps_surprise_qn4,
# 'guid_eps_surprise_qn5':guid_eps_surprise_qn5,
# 'guid_eps_surprise_qn6':guid_eps_surprise_qn6,
# 'guid_eps_surprise_qn7':guid_eps_surprise_qn7,
# 'guid_eps_surprise_qn8':guid_eps_surprise_qn8,
        
   
        'sales_shortfall_guid_q0': sales_shortfall_guid_q0,
        'sales_shortfall_guid_qn1': sales_shortfall_guid_qn1,
        'sales_shortfall_guid_qn2': sales_shortfall_guid_qn2,
        'sales_shortfall_guid_qn3': sales_shortfall_guid_qn3,
        'sales_shortfall_guid_qn4': sales_shortfall_guid_qn4,
        'sales_shortfall_guid_qn5': sales_shortfall_guid_qn5,
        'sales_shortfall_guid_qn6': sales_shortfall_guid_qn6,
        'sales_shortfall_guid_qn7': sales_shortfall_guid_qn7,
        'sales_shortfall_guid_qn8': sales_shortfall_guid_qn8,
  
            
        
        'fq1_eps_std': fq1_eps_std,
#         'fq1_eps_std_exsec': fq1_eps_std_exsec,
        'fq1_eps_up':fq1_eps_up,
        'fq1_eps_dwn':fq1_eps_dwn,
        'fq1_eps_num_est':fq1_eps_num_est,
        'fq1_sales_mean': fq1_sales_mean,
        'fq1_sales_std': fq1_sales_std,

        'guid_eps_surprise': guid_eps_surprise_q0,
        
        #Consensus
#         'fq0_sales_cons': fq0_sales_cons,
# 'fqn1_sales_cons': fqn1_sales_cons,
# 'fqn2_sales_cons': fqn2_sales_cons,
# 'fqn3_sales_cons': fqn3_sales_cons,
# 'fqn4_sales_cons': fqn4_sales_cons,
# 'fqn5_sales_cons': fqn5_sales_cons,
# 'fqn6_sales_cons': fqn6_sales_cons,
# 'fqn7_sales_cons': fqn7_sales_cons,
# 'fqn8_sales_cons': fqn8_sales_cons,
#         'fqn9_sales_cons': fqn9_sales_cons,
#         'fqn10_sales_cons': fqn10_sales_cons,
        
        'fq0_eps_cons': fq0_eps_cons,
'fqn1_eps_cons': fqn1_eps_cons,
'fqn2_eps_cons': fqn2_eps_cons,
'fqn3_eps_cons': fqn3_eps_cons,
'fqn4_eps_cons': fqn4_eps_cons,
'fqn5_eps_cons': fqn5_eps_cons,
'fqn6_eps_cons': fqn6_eps_cons,
'fqn7_eps_cons': fqn7_eps_cons,
'fqn8_eps_cons': fqn8_eps_cons,
#         'fqn9_eps_cons': fqn9_eps_cons,
#         'fqn10_eps_cons': fqn10_eps_cons,

        #CONSISTENCY WORKS
        'fq0_sales_surprise': fq0_sales_surprise,
        'fqn1_sales_surprise': fqn1_sales_surprise,
        'fqn2_sales_surprise': fqn2_sales_surprise,
        'fqn3_sales_surprise': fqn3_sales_surprise,
        'fqn4_sales_surprise': fqn4_sales_surprise,
        'fqn5_sales_surprise': fqn5_sales_surprise,
        'fqn6_sales_surprise': fqn6_sales_surprise,
        'fqn7_sales_surprise': fqn7_sales_surprise,
        'fqn8_sales_surprise': fqn8_sales_surprise,
        
# next quarter guidance = guidance q1 - actuals q0
# next quarter guidance = guidance q1 - actuals q0 * growth from qn1 to q0
        
        
        #GUIDANCE
        
'fq0_sales_guid': fq0_sales_guid,
'fqn1_sales_guid': fqn1_sales_guid,
'fqn2_sales_guid': fqn2_sales_guid,
'fqn3_sales_guid': fqn3_sales_guid,
'fqn4_sales_guid': fqn4_sales_guid,
'fqn5_sales_guid': fqn5_sales_guid,
'fqn6_sales_guid': fqn6_sales_guid,
'fqn7_sales_guid': fqn7_sales_guid,
'fqn8_sales_guid': fqn8_sales_guid,
#         'fqn9_sales_guid': fqn9_sales_guid,
#         'fqn10_sales_guid': fqn10_sales_guid,
        
#         'fq0_eps_guid': fq0_eps_guid,
# 'fqn1_eps_guid': fqn1_eps_guid,
# 'fqn2_eps_guid': fqn2_eps_guid,
# 'fqn3_eps_guid': fqn3_eps_guid,
# 'fqn4_eps_guid': fqn4_eps_guid,
# 'fqn5_eps_guid': fqn5_eps_guid,
# 'fqn6_eps_guid': fqn6_eps_guid,
# 'fqn7_eps_guid': fqn7_eps_guid,
# 'fqn8_eps_guid': fqn8_eps_guid,
#         'fqn9_eps_guid': fqn9_eps_guid,
#         'fqn10_eps_guid': fqn10_eps_guid,


'fq0_sales_surprise_guid':fq0_sales_surprise_guid,  
'fqn1_sales_surprise_guid':fqn1_sales_surprise_guid,
'fqn2_sales_surprise_guid':fqn2_sales_surprise_guid,
'fqn3_sales_surprise_guid':fqn3_sales_surprise_guid,
'fqn4_sales_surprise_guid':fqn4_sales_surprise_guid,
'fqn5_sales_surprise_guid':fqn5_sales_surprise_guid,
'fqn6_sales_surprise_guid':fqn6_sales_surprise_guid,
'fqn7_sales_surprise_guid':fqn7_sales_surprise_guid,
'fqn8_sales_surprise_guid':fqn8_sales_surprise_guid,
        
#                'fq0_sales_yield_cons':fq0_sales_yield_cons,
#                 'fqn1_sales_yield_cons':fqn1_sales_yield_cons,
#                 'fqn2_sales_yield_cons':fqn2_sales_yield_cons,
#                 'fqn3_sales_yield_cons':fqn3_sales_yield_cons,
#                 'fqn4_sales_yield_cons':fqn4_sales_yield_cons,
#                 'fqn5_sales_yield_cons':fqn5_sales_yield_cons,
#                 'fqn6_sales_yield_cons':fqn6_sales_yield_cons,
#                         'fqn7_sales_yield_cons':fqn7_sales_yield_cons,
#                         'fqn8_sales_yield_cons':fqn8_sales_yield_cons,
        
            
    },
    screen=base_universe
)

output = run_pipeline(pipe_my_alpha_factor, '2015-06-01', '2018-09-01')


# Market and Volatility Factors
# mk = transform(output, 'log_mkt_cap', 1)
# mk_inv = transform(output, 'log_mkt_cap', -1)
# output['vol_rank'] = signalize(output['vol'])
# output['mk_rank'] = signalize(output['mkt_cap']**(1.0/3.0))

# /print output['sales_shortfall_guid_qn8']

In [ ]:
# W
alpha1 = signalize(1/output['fq1_eps_std']).dropna()
# alpha1exs = signalize(1/output['fq1_eps_std_exsec']).dropna()

#W
alpha2 = signalize((output['fq1_eps_up'] - output['fq1_eps_dwn'])/output['fq1_eps_num_est']).dropna()
#W
alpha3 = signalize(output['fq1_sales_mean']/output['fq1_sales_std']).dropna()

#W
alpha9 = signalize(output['guid_eps_surprise']).dropna()

alpha_s1 = alpha1 + alpha2 + alpha3 + alpha9 

sales_surprise_std = output[[
    'fq0_sales_surprise',
    'fqn1_sales_surprise', 
    'fqn2_sales_surprise',
    'fqn3_sales_surprise',
    'fqn4_sales_surprise',
    'fqn5_sales_surprise',
    'fqn6_sales_surprise',
    'fqn7_sales_surprise',
    'fqn8_sales_surprise'
]].std(axis=1)

sales_surprise_mean = output[[
    'fq0_sales_surprise',
    'fqn1_sales_surprise', 
    'fqn2_sales_surprise',
    'fqn3_sales_surprise',
    'fqn4_sales_surprise',
    'fqn5_sales_surprise',
    'fqn6_sales_surprise',
    'fqn7_sales_surprise',
    'fqn8_sales_surprise'
]].mean(axis=1)

alpha38 = signalize(sales_surprise_mean/sales_surprise_std)

sales_surprise_guid_std = output[[
    'fq0_sales_surprise_guid',
    'fqn1_sales_surprise_guid', 
    'fqn2_sales_surprise_guid',
    'fqn3_sales_surprise_guid',
    'fqn4_sales_surprise_guid',
    'fqn5_sales_surprise_guid',
    'fqn6_sales_surprise_guid',
    'fqn7_sales_surprise_guid',
    'fqn8_sales_surprise_guid'
]].std(axis=1)

sales_surprise_guid_mean = output[[
    'fq0_sales_surprise_guid',
    'fqn1_sales_surprise_guid', 
    'fqn2_sales_surprise_guid',
    'fqn3_sales_surprise_guid',
    'fqn4_sales_surprise_guid',
    'fqn5_sales_surprise_guid',
    'fqn6_sales_surprise_guid',
    'fqn7_sales_surprise_guid',
    'fqn8_sales_surprise_guid'
]].mean(axis=1)

alpha39 = signalize(sales_surprise_guid_mean/sales_surprise_guid_std)


cs_sales_mean_q = output[['fq0_sales_guid', 
'fqn1_sales_guid',
'fqn2_sales_guid',
'fqn3_sales_guid',
'fqn4_sales_guid',
'fqn5_sales_guid',
'fqn7_sales_guid',
'fqn8_sales_guid']].mean(axis=1)

cs_sales_std_q = output[['fq0_sales_guid', 
'fqn1_sales_guid',
'fqn2_sales_guid',
'fqn3_sales_guid',
'fqn4_sales_guid',
'fqn5_sales_guid',
'fqn7_sales_guid',
'fqn8_sales_guid']].std(axis=1)

#W
alpha34 = signalize(cs_sales_mean_q/cs_sales_std_q)

alpha361 = signalize((output['fq0_sales_guid'] - output['fqn3_sales_guid'])/output['fqn3_sales_guid'])
alpha362 = signalize((output['fq0_sales_guid'] - output['fqn2_sales_guid'])/output['fqn2_sales_guid'])
alpha36 = (alpha361+alpha362)/2

alpha_s1 = alpha_s1.sub(alpha_s1.mean(level=0), level=0)

# cs_sales_yield_mean_q = output[['fq0_sales_yield_cons', 
# 'fqn1_sales_yield_cons',
# 'fqn2_sales_yield_cons',
# 'fqn3_sales_yield_cons',
# 'fqn4_sales_yield_cons',
# 'fqn5_sales_yield_cons',
# 'fqn7_sales_yield_cons',
# 'fqn8_sales_yield_cons']].mean(axis=1)

# cs_sales_yield_std_q = output[['fq0_sales_yield_cons', 
# 'fqn1_sales_yield_cons',
# 'fqn2_sales_yield_cons',
# 'fqn3_sales_yield_cons',
# 'fqn4_sales_yield_cons',
# 'fqn5_sales_yield_cons',
# 'fqn7_sales_yield_cons',
# 'fqn8_sales_yield_cons']].std(axis=1)

# alpha40 = signalize(cs_sales_yield_mean_q/cs_sales_yield_std_q)


cs_sales_guidance_growth_mean = output[['sales_shortfall_guid_q0', 
                                        'sales_shortfall_guid_qn1',
                                       'sales_shortfall_guid_qn2',
                                        'sales_shortfall_guid_qn3',
                                        'sales_shortfall_guid_qn4',
                                        'sales_shortfall_guid_qn5',
                                        'sales_shortfall_guid_qn6',
                                        'sales_shortfall_guid_qn7',
                                        'sales_shortfall_guid_qn8'
                                       ]].mean(axis=1) 

cs_sales_guidance_growth_std = output[['sales_shortfall_guid_q0', 
                                        'sales_shortfall_guid_qn1',
                                       'sales_shortfall_guid_qn2',
                                        'sales_shortfall_guid_qn3',
                                        'sales_shortfall_guid_qn4',
                                        'sales_shortfall_guid_qn5',
                                        'sales_shortfall_guid_qn6',
                                        'sales_shortfall_guid_qn7',
                                        'sales_shortfall_guid_qn8'
                                       ]].std(axis=1) 


alpha411 = signalize(cs_sales_guidance_growth_mean)
alpha412 = signalize(-cs_sales_guidance_growth_std)
alpha413 = signalize(cs_sales_guidance_growth_mean/cs_sales_guidance_growth_std)



#Sales Suprise interacted with efficiency

efficiency = signalize(signalize(-output['log_mkt_cap']) + signalize(output['vol']) + signalize(-output['fq1_eps_num_est']))

# cs_guid_cons_eps_surprise_mean = output[['guid_eps_surprise_q1',
#                                          'guid_eps_surprise_q0',
#                                          'guid_eps_surprise_qn1',
#                                          'guid_eps_surprise_qn2',
#                                          'guid_eps_surprise_qn3',
#                                          'guid_eps_surprise_qn4',
#                                          'guid_eps_surprise_qn5',
#                                          'guid_eps_surprise_qn6',
#                                           'guid_eps_surprise_qn7',
#                                           'guid_eps_surprise_qn8'
#                                          ]].mean(axis=1)

# cs_guid_cons_eps_surprise_std = output[['guid_eps_surprise_q1',
#                                          'guid_eps_surprise_q0',
#                                          'guid_eps_surprise_qn1',
#                                          'guid_eps_surprise_qn2',
#                                          'guid_eps_surprise_qn3',
#                                          'guid_eps_surprise_qn4',
#                                          'guid_eps_surprise_qn5',
#                                          'guid_eps_surprise_qn6',
#                                           'guid_eps_surprise_qn7',
#                                           'guid_eps_surprise_qn8'
#                                          ]].std(axis=1)


# alpha421 = signalize(cs_guid_cons_eps_surprise_mean) 
# alpha422 = signalize(-cs_guid_cons_eps_surprise_std) 
# alpha423 = signalize(cs_guid_cons_eps_surprise_mean/cs_guid_cons_eps_surprise_std) 
# alpha421e = signalize(alpha421*efficiency)
# alpha422e = signalize(alpha422*efficiency)
# alpha423e = signalize(alpha423*efficiency)


cs_guid_cons_sales_surprise_mean = output[['guid_sales_surprise_q1',
                                         'guid_sales_surprise_q0',
                                         'guid_sales_surprise_qn1',
                                         'guid_sales_surprise_qn2',
                                         'guid_sales_surprise_qn3',
                                         'guid_sales_surprise_qn4',
                                         'guid_sales_surprise_qn5',
                                         'guid_sales_surprise_qn6',
                                          'guid_sales_surprise_qn7'
                                         ]].mean(axis=1)

cs_guid_cons_sales_surprise_std = output[['guid_sales_surprise_q1',
                                         'guid_sales_surprise_q0',
                                         'guid_sales_surprise_qn1',
                                         'guid_sales_surprise_qn2',
                                         'guid_sales_surprise_qn3',
                                         'guid_sales_surprise_qn4',
                                         'guid_sales_surprise_qn5',
                                         'guid_sales_surprise_qn6',
                                          'guid_sales_surprise_qn7'
                                         ]].std(axis=1)


alpha431 = signalize(cs_guid_cons_sales_surprise_mean) 
alpha432 = signalize(-cs_guid_cons_sales_surprise_std) 
alpha433 = signalize(cs_guid_cons_sales_surprise_mean/cs_guid_cons_sales_surprise_std) 
alpha431e = signalize(alpha431*efficiency)
alpha432e = signalize(alpha432*efficiency)
alpha433e = signalize(alpha433*efficiency)


In [ ]:
# print efficiency.head()
print alpha421.head()

In [ ]:
# print alpha41.head()
output['alpha1'] = alpha1
output['alpha2'] = alpha2
output['alpha3'] = alpha3
output['alpha9'] = alpha9
output['alpha38'] = alpha38
output['alpha39'] = alpha39
output['alpha34'] = alpha34
output['alpha36'] = alpha36
# output['alpha40'] = alpha40
output['alpha411'] = alpha411
output['alpha412'] = alpha412
output['alpha413'] = alpha413
# output['alpha421'] = alpha421
# output['alpha422'] = alpha422
# output['alpha423'] = alpha423
# output['alpha424'] = alpha424

output['alpha431'] = alpha431
output['alpha432'] = alpha432
output['alpha433'] = alpha433



# output['weights'] = 1 + 0.3*output['vol_rank'] + 0.7*output['mk_rank']
# print output['alpha411']


In [ ]:
def wmean(name):
    return (output[name]*output['weights']).sum(level=0)/output['weights'].sum(level=0)

alphas = pd.DataFrame()

for name in ['alpha1','alpha2','alpha3','alpha9','alpha38','alpha39',
             'alpha34','alpha36', 'alpha411', 'alpha412', 'alpha413',
#              'alpha421', 'alpha422', 'alpha423','alpha424'
            'alpha431', 'alpha432', 'alpha433']:
    alphas[name] = output[name].sub(output[name].mean(level=0), level=0)
#     alphas[name] = output[name].sub(wmean(name), level=0)

# alphas['weights'] = output['weights']    

In [ ]:
assets = output.index.levels[1].unique()
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
pricing = get_pricing(assets, start_date='2015-06-01', end_date='2018-09-01', fields='close_price')
print "DONE Pricing"


In [ ]:
returns = pricing.pct_change(periods=5).shift(-5)
returns = returns.sub(returns.mean(axis=1), axis=0).stack()
# r_m = returns.mean(level=0)
# print returns

data = pd.concat([returns.rename('returns'), alphas], axis=1).dropna()

data.reset_index(inplace=True)
data = data.rename(columns={'level_0':'dates', 'level_1': 'sid'})
# print data.head()

In [ ]:
data['d1'] = data['alpha1'] 
data['d2'] = data['alpha1'] + data['alpha2'] 
data['d3'] = data['alpha1'] + data['alpha2'] + data['alpha3'] 
data['d4'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38']
data['d5'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] 
# data['d6'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha40']
data['d7'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']
data['d8'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha36']
# data['d9'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha40']
data['d10'] = data['alpha1'] + data['alpha2'] + 2*data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']
data['d11'] = data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha412']
data['d12'] = data['d8'] + data['alpha412'] 
data['d13'] = data['d8'] + data['alpha412'] + data['alpha432'] 
est = sm.OLS(
    100*data[['returns']], 
#             data[['d13']])
        data[['d8', 'alpha413', 'alpha432']])
est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()


In [ ]:
print data['alpha411'].corr(data['alpha412'])

In [ ]:
data['d1'] = data['alpha1'] 
data['d2'] = data['alpha1'] + data['alpha2'] 
data['d3'] = data['alpha1'] + data['alpha2'] + data['alpha3'] 
data['d4'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38']
data['d5'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] 
# data['d6'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha40']
data['d7'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']
data['d8'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha36']
# data['d9'] = data['alpha1'] + data['alpha2'] + data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha40']
data['d10'] = data['alpha1'] + data['alpha2'] + 2*data['alpha3'] + data['alpha38'] + data['alpha39'] + data['alpha34']
data['d11'] = data['alpha38'] + data['alpha39'] + data['alpha34'] + data['alpha412']
data['d12'] = data['d8'] + data['alpha412'] 
est = sm.OLS(
    100*data[['returns']], 
#             data[['d8']])
        data[['d8', 'alpha413', 'alpha432']])
est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()


In [ ]:
# coef =  reg.coef_
# print est2.params
# net_alpha =  data[['alpha1', 'alpha2', 'alpha3', 'alpha9', 'alpha38']].dot(est2.params.T)/np.sum(np.abs(est2.params.T))
# net_alpha = alpha_s1 + alpha38 + alpha39 + alpha34 + alpha36 + alpha40 
# net_alpha = alpha_s1 - alpha9 + alpha38 + alpha39 + alpha40 
net_alpha = alpha_s1 + alpha38 + alpha39 + alpha34 + alpha36 + alpha412
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)